In [ ]:
import tensorflow as tf
from keras.datasets import imdb
from keras.utils import pad_sequences

import os 
import numpy as np

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [ ]:
path_to_file

'/root/.keras/datasets/shakespeare.txt'

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

KeyboardInterrupt: ignored

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [ ]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 6.5990896e-03 -8.0491661e-04  2.9588069e-03 ...  1.7123790e-03
    4.9040182e-03  8.1748265e-04]
  [ 2.9765735e-03  2.8438934e-03 -1.7314260e-03 ...  1.8159207e-03
    5.7474291e-04  1.5131948e-03]
  [ 8.3435391e-04 -8.3201716e-04  2.9813726e-03 ... -1.1281332e-03
    1.9498751e-03 -1.3327150e-04]
  ...
  [ 6.7952625e-03 -1.5051133e-03  5.0865868e-03 ...  5.4934071e-03
   -8.4502632e-03 -3.2617158e-04]
  [ 6.1084516e-05  6.2189263e-04  7.4191783e-03 ...  5.2488940e-03
   -4.2562382e-03 -3.0917411e-03]
  [-1.5618687e-03  2.3696423e-03  2.1944353e-03 ...  5.2982434e-03
   -7.8450162e-03 -2.3223278e-03]]

 [[-4.1093596e-04 -3.6234241e-03 -1.9878540e-03 ...  2.6713074e-03
    2.4149560e-03 -1.1510878e-03]
  [-2.1525826e-03 -6.1838338e-03  3.4255113e-03 ... -1.7863679e-03
    3.2194802e-03 -3.1873665e-03]
  [-2.4450489e-03 -1.5695164e-03 -1.2125322e-03 ...  3.1254345e-04
   -1.7675369e-03 -9.5481437e-04]
  ...
  [ 6.0572159e-03 -1.2118630e-02  1.0903649e-02 ...  6.4004436e

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 6.5990896e-03 -8.0491661e-04  2.9588069e-03 ...  1.7123790e-03
   4.9040182e-03  8.1748265e-04]
 [ 2.9765735e-03  2.8438934e-03 -1.7314260e-03 ...  1.8159207e-03
   5.7474291e-04  1.5131948e-03]
 [ 8.3435391e-04 -8.3201716e-04  2.9813726e-03 ... -1.1281332e-03
   1.9498751e-03 -1.3327150e-04]
 ...
 [ 6.7952625e-03 -1.5051133e-03  5.0865868e-03 ...  5.4934071e-03
  -8.4502632e-03 -3.2617158e-04]
 [ 6.1084516e-05  6.2189263e-04  7.4191783e-03 ...  5.2488940e-03
  -4.2562382e-03 -3.0917411e-03]
 [-1.5618687e-03  2.3696423e-03  2.1944353e-03 ...  5.2982434e-03
  -7.8450162e-03 -2.3223278e-03]], shape=(100, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 0.00659909 -0.00080492  0.00295881 -0.00318686  0.00218056  0.00967585
  0.00321764  0.00388249 -0.00281956  0.00482339 -0.00733554  0.00316856
  0.00023008 -0.00033126  0.00122418 -0.00654856  0.00133901  0.00220707
 -0.0032013  -0.00581087  0.00473592  0.00351033  0.00228146  0.00397234
 -0.00267837  0.00205467  0.00542691  0.00246941  0.00104776  0.00373582
  0.00023682  0.00311069 -0.001994   -0.00501615 -0.00324747 -0.00028087
 -0.00308091 -0.00493471 -0.00173699 -0.00037338  0.00295488 -0.00144024
 -0.0039221   0.00599922 -0.00083556 -0.00654565  0.00252875  0.00233963
 -0.00315632 -0.0013314  -0.00317388 -0.00404066  0.00187268  0.00082768
 -0.00262197 -0.00258375 -0.00451852 -0.00050195 -0.00043584  0.00650063
  0.00297665  0.01059148  0.00171238  0.00490402  0.00081748], shape=(65,), dtype=float32)


In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'IH,fhoVHPW.NbT-jCb$Dj uCOBXpQ;.S$OU!zmvxWKLe;FPhRcByvqqcFtpwVg!o?HB$D$Iyg;KMtqYfj!vWmaSH&GqmPCZppAVL'

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 14s 63ms/step - loss: 2.7200
Epoch 2/50
172/172 [==============================] - 12s 65ms/step - loss: 2.0339
Epoch 3/50
172/172 [==============================] - 12s 64ms/step - loss: 1.7779
Epoch 4/50
172/172 [==============================] - 14s 65ms/step - loss: 1.6291
Epoch 5/50
172/172 [==============================] - 12s 65ms/step - loss: 1.5351
Epoch 6/50
172/172 [==============================] - 12s 65ms/step - loss: 1.4702
Epoch 7/50
172/172 [==============================] - 13s 66ms/step - loss: 1.4248
Epoch 8/50
172/172 [==============================] - 13s 67ms/step - loss: 1.3872
Epoch 9/50
172/172 [==============================] - 13s 67ms/step - loss: 1.3560
Epoch 10/50
172/172 [==============================] - 13s 68ms/step - loss: 1.3300
Epoch 11/50
172/172 [==============================] - 13s 68ms/step - loss: 1.3050
Epoch 12/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2816
E

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights("./training_checkpoints/ckpt_" + str(checkpoint_num))
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeoth, yet, must I lose the ceptreching
rat, corruption'd herdors have fall on her heart
A thought belong up's off the hopely and motion at kings
Pirg into this bexate,
Deteniers thy under hebe
Our fetch hands.

PRINCE EDWARD:
Well and sit and in the Taway. Nor of an afterut?
On we begin there.

QUEEN ELIZABETH:
Play's, sir, let me never title: no minion, know
as he do lapeful throngs the To come:
This arm let as fair wast this till leich ret her son may at marred thy device,
That question after spools with unp the poor Sis
Richamponion, sir, me how you must fellow and!
Now, it shall be on his supposses?
We are thy nobicious.

LICINES:
My lord and Warwick?

GREMIO:
Thou shalt not speak the thing voucheaties of serveal,
If little hand.

Servant:
My vialent, naildo's gracious lady:
IN Dose and 


In [ ]:
!ls


sample_data  training_checkpoints


In [ ]:
!cd training_checkpoints


In [ ]:
ls

sample_data/  training_checkpoints/


In [ ]:
cd

/root
